In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier

from keras import backend as K
from keras.layers import *
from keras.layers import Dense,Flatten,Dropout,Input
from keras.optimizers import RMSprop,Adam
from keras.utils import to_categorical
from keras.models import Model
import tensorflow as tf

In [3]:
#Importing
# train_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/training_v2.csv')
# test_data =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/unlabeled.csv')

# sub_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/samplesubmission.csv')
# data_dict = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [4]:
df_train = np.load('/Users/s0c02nj/Downloads/train_features.npy')
df_test = np.load('/Users/s0c02nj/Downloads/test_features.npy')

In [5]:
df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

In [7]:
df_train.shape

(91713, 1536)

In [ ]:
# cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string'])]['Variable Name'])
# cat_var.remove('icu_admit_type')
# #cat_var.remove('bmi')
# cat_var = cat_var[1:]

In [ ]:
# cat_var  = [x for x in train_data.columns if train_data[x].dtype == 'object' ]
# cont_var = [x for x in train_data.columns if train_data[x].dtype != 'object' ]

In [ ]:
#cont_var = list(data_dict[data_dict['Data Type'].isin(['integer','numeric'])]['Variable Name'])
#cont_var.remove('pred')
#cont_var = cont_var + ['bmi']

In [9]:
#x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [ ]:
x_test = test_data.drop('hospital_death',axis=1)

In [ ]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [ ]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] = x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [ ]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [ ]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

In [ ]:
new_cont_var = list(set(cont_var).intersection(set(x_comb.columns)))

In [ ]:
for col in tqdm(new_cont_var):
    x_comb[col] = x_comb[col].fillna(x_comb[col].mean())

In [ ]:
count_cols = []

for i,col in tqdm(enumerate(new_cat_var + new_cont_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_cols.append(str(col)+'count')

In [ ]:
total_cat_cols = new_cat_var + count_cols

In [ ]:
x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [ ]:
new_cont_var = list(set(cont_var).intersection(set(x_comb.columns)))

In [ ]:
ls_rem = []
for col in tqdm(new_cont_var):
    if x_comb[col].std() <0.01 :
        ls_rem.append(col)

In [ ]:
ls_rem

In [ ]:
x_comb = x_comb.drop(['readmission_status'],axis=1)

In [ ]:
new_cont_var.remove('readmission_status')

In [ ]:
#total_cont = new_cont_var + count_cols

In [ ]:
for col in tqdm(new_cont_var):
    mu = x_comb[col].mean()
    std = x_comb[col].std()
    x_comb[col] = x_comb[col].apply(lambda x : (x-mu)/std)

In [ ]:
train_x = x_comb[0:91713]
test_x =  x_comb[91713:]

In [ ]:
train_cat = train_x[total_cat_cols]
test_cat =  test_x[total_cat_cols]

train_cont = train_x[new_cont_var]
test_cont = test_x[new_cont_var]

In [ ]:
#len(total_cat_cols) + len(new_cont_var)
def get_train_test_data_cat(df):
    
    data=[]
    for col in total_cat_cols:
        data.append(df[col])
    
    return data

In [ ]:
x_train_cat = get_train_test_data_cat(train_x)
x_test_cat  = get_train_test_data_cat(test_x)

In [ ]:
x_train_comb = x_train_cat + [train_cont]
x_test_comb =  x_test_cat +  [test_cont]

In [ ]:
y_cat = to_categorical(y_train)

In [ ]:
len(total_cat_cols) 

len(new_cont_var)

In [ ]:
#len(total_cat_cols) + len(new_cont_var)
def model_deep():
    
    layer_cat  = []
    input_cat =  []
    
    #Categorical_var
    for i,cat_var in tqdm(enumerate((total_cat_cols))): 
        
        no_of_unique_cat  = x_comb[cat_var].nunique()
        embedding_size = min(np.ceil((no_of_unique_cat)/2),30)
        embedding_size = int(embedding_size)
        
        #Defining the input-----> branch_id
        input_catg =  Input(shape=(1,))
        layer_catg =  Embedding(no_of_unique_cat+1 ,embedding_size,input_length=1,trainable=True)(input_catg)
        layer_catg =  Flatten()(layer_catg)
        layer_cat.append(layer_catg)
        input_cat.append(input_catg)
    
    
    #continuous var
    input_cont = Input(shape=(174,))
    layer_cont = Dense(80, activation='relu')(input_cont)
        
    #Merging
    layer_comb =  layer_cat + [layer_cont]
    layer_comb = concatenate(layer_comb)
    
    #Layers
    layer_dense = Dense(200, activation='relu')(layer_comb)
    layer_dense = Dropout(0.5)(layer_dense)
    
    layer_dense = Dense(100, activation='relu')(layer_comb)
    layer_dense = Dropout(0.5)(layer_dense)
    
    layer_dense = Dense(30, activation='relu')(layer_comb)
    layer_dense = Dropout(0.5)(layer_dense)
    
    #Final output
    layer_output = Dense(2,activation='softmax')(layer_dense)
    
    #Comb_inpus
    input_comb = input_cat + [input_cont]
    
    #Final model
    model = Model(inputs= input_comb ,outputs=layer_output)
    
    return model

In [ ]:
deep_model = model_deep()
deep_model.summary()

In [ ]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [ ]:
deep_model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr=0.0004), metrics = [auc])

In [ ]:
history = deep_model.fit(x_train_comb,y_cat,
                         batch_size = 512, 
                         epochs = 10, 
                         class_weight= [1,10],
                         verbose = 1)

In [ ]:
# def model_deep():
    
#     #Defining the input-----> Transaction
#     inputs1 = Input(shape=(x1_train.shape[1],))
#     layer_dense = Dense(80, activation='relu')(inputs1)
    
#     layer_dense = Dense(50, activation='tanh')(layer_dense)
#     #layer_dense = Dropout(0.3)(layer_dense)
    
#     layer_dense = Dense(25, activation='relu')(layer_dense)
#     #layer_dense = Dropout(0.3)(layer_dense)
    
#     layer_dense = Dense(10, activation='tanh')(layer_dense)
#     #layer_dense = Dropout(0.2)(layer_dense)
    
#     layer_dense = Dense(5, activation='relu')(layer_dense)
#     #layer_dense = Dropout(0.2)(layer_dense)
    
#     probabilities = Dense(2,activation='softmax')(layer_dense)
    
#     model = Model(inputs= inputs1,outputs = probabilities)
#     return model


In [ ]:
# history = model.fit(x1_train, y1_train,
#                          batch_size = 512, 
#                          epochs = 10,
#                          class_weight= [{0:1,1:10}],
#                          validation_data = (x1_val,y1_val), 
#                          verbose = 1)

In [ ]:
# x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
#                                                       test_size=0.2,
#                                                       random_state=0,
#                                                       stratify = y_train)

In [ ]:
# def create_model(data, catcols,contcols):   
    
#     #Categorical Variable
#     inputs_cat = []
#     outputs_cat = []
#     for c in catcols:
#         num_unique_values = int(data[c].nunique())
#         embed_dim = int(min(np.ceil((num_unique_values)/2), 50))
#         inp = layers.Input(shape=(1,))
#         out = layers.Embedding(num_unique_values + 1, embed_dim, name=c)(inp)
#         out = layers.SpatialDropout1D(0.3)(out)
#         out = layers.Reshape(target_shape=(embed_dim, ))(out)
#         inputs_cat.append(inp)
#         outputs_cat.append(out)
    
#     x_cat = layers.Concatenate()(outputs_cat)
#     x_cat = layers.BatchNormalization()(x_cat)
    
#     x_cat = layers.Dense(200, activation="relu")(x_cat)
#     x_cat = layers.Dropout(0.3)(x_cat)
#     x_cat = layers.BatchNormalization()(x_cat)
    
#     x_cat = layers.Dense(100, activation="relu")(x_cat)
#     x_cat = layers.Dropout(0.3)(x_cat)
#     x_cat = layers.BatchNormalization()(x_cat)
    
    
#     #Continuous Variable
#     inputs_cont = layers.Input(shape=(data[contcols].shape[1],))
#     x_cont = layers.Dense(200, activation="relu")(inputs_cont)
#     x_cont = layers.BatchNormalization()(x_cont)
    
#     #Merge,Cocatenating the inputes
#     x_merge = layers.Concatenate()([x_cont, x_cat])
#     x_merge = layers.Dense(200, activation="relu")(x_merge)
#     x_merge = layers.Dropout(0.4)(x_merge)
#     x_merge = layers.BatchNormalization()(x_merge)
    
#     #Output
#     y = layers.Dense(1, activation="softmax")(x_merge)
    
#     #Comb_inpus
#     inputs_comb = inputs_cat + [inputs_cont]

#     #Final model
#     model = Model(inputs= inputs_comb ,outputs = y)
    
#     return model

In [ ]:
#deep_model = create_model(x_comb, new_cat_var,new_cont_var)

In [ ]:
#deep_model.summary()

In [ ]:
# def auc(y_true, y_pred):
#     def fallback_auc(y_true, y_pred):
#         try:
#             return metrics.roc_auc_score(y_true, y_pred)
#         except:
#             return 0.5
#     return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [ ]:
#deep_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])

In [ ]:
# history = deep_model.fit(x_train_comb,y_train,
#                          batch_size = 512, 
#                          epochs = 100, 
#                          validation_split = 0.2, 
#                          class_weight= [1,10],
#                          verbose = 1)